# Fake News Detector

## Installing Necessary Libraries

In [41]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [42]:
import os
print(os.getcwd())


c:\Users\user\Desktop\pro\Fake-News-Detection


In [43]:
data_fake = pd.read_csv('C:/Users/user/Fake.csv', encoding='ISO-8859-1')
data_true = pd.read_csv('C:/Users/user/True.csv', encoding='ISO-8859-1')
manual_testing = pd.read_csv('C:/Users/user/manual_testing.csv', encoding='ISO-8859-1')


C:\Users\user\AppData\Local\Temp\ipykernel_2356\2576768497.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data_fake = pd.read_csv('C:/Users/user/Fake.csv', encoding='ISO-8859-1')


## Loading the data

In [44]:
data_fake_manual_testing = pd.read_csv(r"C:\Users\user\Fake.csv", encoding='ISO-8859-1')
data_true_manual_testing = pd.read_csv(r"C:\Users\user\True.csv", encoding='ISO-8859-1')


C:\Users\user\AppData\Local\Temp\ipykernel_2356\295258509.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data_fake_manual_testing = pd.read_csv(r"C:\Users\user\Fake.csv", encoding='ISO-8859-1')


### Data Preview 

In [45]:
data_fake.head()

,title,text,subject,date,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 162,Unnamed: 163,Unnamed: 164,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168,Unnamed: 169,Unnamed: 170,Unnamed: 171
0,Donald Trump Sends Out Embarrassing New Yearâ...,Donald Trump just couldn t wish all Americans ...,News,31-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,31-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,30-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Trump Is So Obsessed He Even Has Obamaâs Na...,"On Christmas day, Donald Trump announced that ...",News,29-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,25-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
data_true.tail()

,title,text,subject,date
41794,Prez Trump to be hospitalised for next ï¿½few ...,Prez Trump to be hospitalised for next ï¿½few ...,NaN,20-Mar
41795,COVID-19 positive Malaika Arora assures fans s...,COVID-19 positive Malaika Arora assures fans s...,NaN,17-Mar
41796,COVID-19 | First case of coronavirus reinfecti...,COVID-19 | First case of coronavirus reinfecti...,NaN,20-Jul
41797,Unlock 4.0 guidelines: Know what is allowed an...,Unlock 4.0 guidelines: Know what is allowed an...,NaN,18-Jun
41798,"Now, one of the oldest circuses of the country...","Now, one of the oldest circuses of the country...",NaN,20-Aug


In [47]:
data_fake["class"]=0
data_true['class']=1

In [48]:
data_fake.shape, data_true.shape

((23311, 173), (41799, 5))

In [49]:
# Drop rows in data_fake
for i in range(23480, 23470, -1):
    if i in data_fake.index:  # Check if the index exists
        data_fake.drop([i], axis=0, inplace=True)

# Drop rows in data_true
for i in range(21416, 21406, -1):
    if i in data_true.index:  # Check if the index exists
        data_true.drop([i], axis=0, inplace=True)


In [50]:
data_fake.shape, data_true.shape

((23311, 173), (41789, 5))

In [51]:
data_fake_manual_testing['class']=0
data_true_manual_testing['class']=1

In [52]:
data_fake_manual_testing.head(10)

,title,text,subject,date,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 163,Unnamed: 164,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168,Unnamed: 169,Unnamed: 170,Unnamed: 171,class
0,Donald Trump Sends Out Embarrassing New Yearâ...,Donald Trump just couldn t wish all Americans ...,News,31-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,31-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,30-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,Trump Is So Obsessed He Even Has Obamaâs Na...,"On Christmas day, Donald Trump announced that ...",News,29-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,25-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,25-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,23-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,23-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,22-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,21-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [53]:
data_true_manual_testing.head(10)

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,31-Dec-17,1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,29-Dec-17,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,31-Dec-17,1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,30-Dec-17,1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,29-Dec-17,1
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,29-Dec-17,1
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,29-Dec-17,1
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statementsÂ were posted to the v...,politicsNews,29-Dec-17,1
8,Trump on Twitter (Dec 28) - Global Warming,The following statementsÂ were posted to the v...,politicsNews,29-Dec-17,1
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,politicsNews,28-Dec-17,1


In [54]:
data_merge=pd.concat([data_fake, data_true], axis = 0)
data_merge.head(10)

,title,text,subject,date,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 163,Unnamed: 164,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168,Unnamed: 169,Unnamed: 170,Unnamed: 171,class
0,Donald Trump Sends Out Embarrassing New Yearâ...,Donald Trump just couldn t wish all Americans ...,News,31-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,31-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,30-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,Trump Is So Obsessed He Even Has Obamaâs Na...,"On Christmas day, Donald Trump announced that ...",News,29-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,25-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,25-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,23-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,23-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,22-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,21-Dec-17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


#### "title",  "subject" and "date" columns is not required for detecting the fake news, so I am going to drop the columns.

In [55]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       ...
       'Unnamed: 163', 'Unnamed: 164', 'Unnamed: 165', 'Unnamed: 166',
       'Unnamed: 167', 'Unnamed: 168', 'Unnamed: 169', 'Unnamed: 170',
       'Unnamed: 171', 'class'],
      dtype='object', length=173)

In [56]:
data=data_merge.drop(['title','subject','date'], axis = 1)

In [57]:
#count of missing values
data.isnull().sum() 

text              401
Unnamed: 4      65100
Unnamed: 5      65100
Unnamed: 6      65100
Unnamed: 7      65100
                ...  
Unnamed: 168    65100
Unnamed: 169    65100
Unnamed: 170    65100
Unnamed: 171    65100
class               0
Length: 170, dtype: int64

#### Randomly shuffling the dataframe 

In [58]:
data = data.sample(frac = 1)

In [59]:
data.head()

,text,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,...,Unnamed: 163,Unnamed: 164,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168,Unnamed: 169,Unnamed: 170,Unnamed: 171,class
19248,"FACT CHECK: No, Bhutan didnt stop irrigation ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
20839,Fact Check: Netizens mistake artwork for cloud...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9912,"Third straight day of record cases in Bengal, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
19238,Last chance for students to correct JEE (Main)...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
16834,"After Pak victory, old video viral claiming yo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [60]:
data.reset_index(inplace = True)
data.drop(['index'], axis = 1, inplace = True)

In [61]:
data.columns

Index(['text', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7',
       'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       ...
       'Unnamed: 163', 'Unnamed: 164', 'Unnamed: 165', 'Unnamed: 166',
       'Unnamed: 167', 'Unnamed: 168', 'Unnamed: 169', 'Unnamed: 170',
       'Unnamed: 171', 'class'],
      dtype='object', length=170)

In [62]:
data.head()

,text,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,...,Unnamed: 163,Unnamed: 164,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168,Unnamed: 169,Unnamed: 170,Unnamed: 171,class
0,"FACT CHECK: No, Bhutan didnt stop irrigation ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Fact Check: Netizens mistake artwork for cloud...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,"Third straight day of record cases in Bengal, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Last chance for students to correct JEE (Main)...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,"After Pak victory, old video viral claiming yo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


## Preprocessing Text

#### Creating a function to convert the text in lowercase, remove the extra space, special chr., ulr and links.

In [63]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]','',text)
    text = re.sub("\\W"," ",text)
    text = re.sub('https?://\S+|www\.\S+','',text)
    text = re.sub('<.*?>+',b'',text)
    text = re.sub('[%s]' % re.escape(string.punctuation),'',text)
    text = re.sub('\w*\d\w*','',text)
    return text

In [64]:
data['text'] = data['text'].astype(str)
data['text'] = data['text'].apply(wordopt)


In [65]:
wordopt(data['text'].iloc[0])


'fact check  no  bhutan didn t stop irrigation water for indian farmers  fact check  has bhutan deliberately blocked irrigation water to indian farmers in assam '

In [66]:
data['text'] = data['text'].apply(wordopt)

#### Defining dependent and independent variable as x and y

In [67]:
x = data['text']
y = data['class']

## Training the model

#### Splitting the dataset into training set and testing set. 

In [68]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.25)

### Extracting Features from the Text

#### Convert text to vectors

In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

## Logistic Regression

In [70]:
from sklearn.linear_model import LogisticRegression

In [71]:
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [72]:
pred_lr = LR.predict(xv_test)

In [73]:
LR.score(xv_test, y_test)

0.9560061443932412

In [74]:
print (classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.98      0.90      0.94      5790
           1       0.94      0.99      0.97     10485

    accuracy                           0.96     16275
   macro avg       0.96      0.94      0.95     16275
weighted avg       0.96      0.96      0.96     16275



## Decision Tree Classifier

In [75]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [76]:
pred_dt = DT.predict(xv_test)

In [77]:
DT.score(xv_test, y_test)

0.9343778801843318

In [78]:
print (classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.98      0.90      0.94      5790
           1       0.94      0.99      0.97     10485

    accuracy                           0.96     16275
   macro avg       0.96      0.94      0.95     16275
weighted avg       0.96      0.96      0.96     16275



## Gradient Boost Classifier

In [79]:
from sklearn.ensemble import GradientBoostingClassifier

GB = GradientBoostingClassifier(random_state = 0)
GB.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [80]:
pred_gb = GB.predict(xv_test)

In [81]:
GB.score(xv_test, y_test)

0.9383717357910907

In [82]:
print(classification_report(y_test, pred_gb))

              precision    recall  f1-score   support

           0       0.98      0.84      0.91      5790
           1       0.92      0.99      0.95     10485

    accuracy                           0.94     16275
   macro avg       0.95      0.92      0.93     16275
weighted avg       0.94      0.94      0.94     16275



## Random Forest Classifier

In [83]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(random_state = 0)
RF.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [84]:
pred_rf = RF.predict(xv_test)

In [85]:
RF.score(xv_test, y_test)

0.9594470046082949

In [86]:
print (classification_report(y_test, pred_rf))

              precision    recall  f1-score   support

           0       0.96      0.92      0.94      5790
           1       0.96      0.98      0.97     10485

    accuracy                           0.96     16275
   macro avg       0.96      0.95      0.96     16275
weighted avg       0.96      0.96      0.96     16275



## Testing the Model

In [87]:
def output_lable(n):
    if n==0:
        return "Fake News"
    elif n==1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test['text'] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GB = GB.predict(new_xv_test)
    pred_RF = RF.predict(new_xv_test)
    
    return print("\n\nLR Predicition: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction:{}".format(output_lable(pred_LR[0]),
                                                                                                             output_lable(pred_DT[0]),
                                                                                                             output_lable(pred_GB[0]),
                                                                                                             output_lable(pred_RF[0])))

### Model Testing With Manual Entry

In [88]:
print("model defined?", 'model' in globals())
print("vectorizer defined?", 'vectorization' in globals())


model defined? False
vectorizer defined? True


In [89]:
def test_news(news_text):
    # Preprocess it like the training data
    processed = wordopt(news_text)
    
    # Vectorize
    vector_input = vectorization.transform([processed])
    
    # Predict
    prediction = model.predict(vector_input)
    
    # Interpret result
    if prediction[0] == 1:
        print("✅ This news is REAL.")
    else:
        print("❌ This news is FAKE.")


In [90]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
import joblib

# Load datasets
# Load datasets with consistent, permissive encoding
data_fake = pd.read_csv('C:/Users/user/Fake.csv', encoding='ISO-8859-1', low_memory=False)
data_true = pd.read_csv('C:/Users/user/True.csv', encoding='ISO-8859-1', low_memory=False)

# Label the data
data_true['label'] = 1  # Real
data_fake['label'] = 0  # Fake

# Combine them
data = pd.concat([data_true, data_fake], ignore_index=True)

# ✅ CLEAN MISSING TEXT
data = data.dropna(subset=['text'])  # <- 🧼 this is the fix

# Select features and labels
X = data['text']
y = data['label']

# Vectorize text
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
X_vec = vectorizer.fit_transform(X)

# Split and train
X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42)
model = PassiveAggressiveClassifier(max_iter=1000)
model.fit(X_train, y_train)

# Save model and vectorizer
joblib.dump(model, 'model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')

print("✅ Model and vectorizer saved as 'model.pkl' and 'vectorizer.pkl'")


✅ Model and vectorizer saved as 'model.pkl' and 'vectorizer.pkl'


In [91]:
import joblib

# Load the saved model and vectorizer
model = joblib.load('model.pkl')
vectorizer = joblib.load('vectorizer.pkl')

# 🔄 Replace the text below with your own news article
sample_news = """Donald Trump thinks if he rage-tweets about NFL players who choose to kneel instead of stand during the national anthem as a form of silent protest against racial injustice and police brutality, it will all go away but by doing that, he inadvertently highlights that there is a problem. The former reality show star dropped a racial slur yesterday in front of Native American heroes, then today, he jumped on Twitter to yell at black people. We see a pattern here. At least 24 players kneeling this weekend at NFL stadiums that are now having a very hard time filling up,  Trump screamed at the world.  The American public is fed up with the disrespect the NFL is paying to our Country, our Flag and our National Anthem. Weak and out of control! At least 24 players kneeling this weekend at NFL stadiums that are now having a very hard time filling up. The American public is fed up with the disrespect the NFL is paying to our Country, our Flag and our National Anthem. Weak and out of control!  Donald J. Trump (@realDonaldTrump) November 28, 2017This didn t go down well on the Internet.Hey Mr. Clumsy, Now that you have added the Native Americans to the long list of Americans you insulted, can you make life easier on us and just tell us when and where you ll be using the  N  word? The suspense is killing us. #disgrace      (@ronenbergen) November 28, 2017Collin Kapernick was named Time person of the year. Way to go Collin #Time #TakeAKnee  Winning MN (@WinningMN) November 28, 2017Let it go. Your just mad because they aren t doing what YOU want them to do.  Rick Laferriere (@RickLaf2) November 28, 2017The American public is fed up with the disrespect the President is paying to blacks, browns, mexicans, gays and all women since he entered office.  Hafiz Shariff (@HafizDoc) November 28, 2017Mr. Trump, you are fake news. https://t.co/7LA9ExaK3b  Leanne (@MsTeacher80) November 28, 2017"Weak and out of control!" Pot calling the kettle black. pic.twitter.com/6ovSNvbStm  Dan (QWERTYGEO) (@The_QWERTYGEO) November 28, 2017Disrespect is being an a-hole while trying to honor code talkers while standing in front of Andrew Jackson.  kimberly (@kdd75) November 28, 2017According to numbers compiled by the Associated Press from its reporters at stadiums around the country on Sunday, it was 23 not 24 players who participated in the protests.As for Trump s claim about a lack of attendance, attendance through Week 12 was listed at nearly 12 million fans for the league s 32 teams by Pro Football Reference, with Week 12 drawing just over 1 million, according to The Washington Post.The average attendance at NFL games was actually up slightly, from 68,914 per game in 2016 to 69,264 per game, so President Liar Pants lied again. If he wants to talk about attendance, we can discuss his unenthusiastic inauguration crowd size. As for his ratings, he ranks as the least popular president in the history of polling.Donald Trump is actively trying to divide our country even more than it already is. Trump s obsession with NFL athletes protesting against racial injustice is obvious, and yet, he didn t seem to be concerned about the tiki-torch carrying Nazis who marched in his name in Charlottesville this year.Trump s outrageous attacks during the past two days mean that some big news is about to drop. Wait for it.Photo by Chip Somodevilla/Getty Images.
"""

# Transform and predict
vectorized_input = vectorizer.transform([sample_news])
prediction = model.predict(vectorized_input)

# 🧠 Output result
print("Prediction:", "REAL" if prediction[0] == 1 else "FAKE")


Prediction: FAKE
